# Import externally 

In [1]:
# import sys
# import os

# # Add the parent directory to the sys.path
# parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# if parent_dir not in sys.path:
#     sys.path.append(parent_dir)

# Load data

In [2]:
import pandas as pd
from constants import NULLABLE_MEASURES, TEMP_PATH
from utils.class_patient import Patients

patients = Patients.loadPatients(False, TEMP_PATH / "learning_data-org.json")
patients.fillMissingMeasureValue(NULLABLE_MEASURES, 0)

# Remove missing data

## Remove features with more than 20% missing

In [3]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


## Remove patients with more than 20% missing features

In [4]:
patients.removePatientByMissingFeatures()
len(patients)

1209

## Display insight

In [5]:
# dfData = patients.getMeasuresBetween(pd.Timedelta(hours=-6), pd.Timedelta(hours=24), "first")

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(patients.getMeasures())

Counter({'age': 1209,
         'chronic_pulmonary_disease': 1209,
         'ckd_stage': 1209,
         'congestive_heart_failure': 1209,
         'dka_type': 1209,
         'gender': 1209,
         'history_aci': 1209,
         'history_ami': 1209,
         'hypertension': 1209,
         'liver_disease': 1209,
         'macroangiopathy': 1209,
         'malignant_cancer': 1209,
         'mechanical_ventilation': 1209,
         'microangiopathy': 1209,
         'oasis': 1209,
         'preiculos': 1209,
         'race': 1209,
         'saps2': 1209,
         'sofa': 1209,
         'use_NaHCO3': 1209,
         'uti': 1209,
         'ag': 1208,
         'bg': 1208,
         'bicarbonate': 1208,
         'bun': 1208,
         'egfr': 1208,
         'hr': 1208,
         'scr': 1208,
         'dbp': 1207,
         'gcs': 1207,
         'gcs_unable': 1207,
         'rr': 1207,
         'sbp': 1207,
         'phosphate': 1204,
         'calcium': 1203,
         'weight': 1191,
         'plt': 

In [6]:
akdCount = sum([p.akdPositive for p in patients.patientList])

akdCount / len(patients)

0.3937138130686518

# Machine learning

In [7]:
from constants import CATEGORICAL_MEASURES


idColumns = ["subject_id", "hadm_id", "stay_id"]
categoryColumns = CATEGORICAL_MEASURES
labelColumn = "akd"

## Split train-test

In [8]:
splitedPatients = patients.split(5, 27)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

## Define model

In [9]:
how = "first"


def createModel():
    import os
    from GRANDE import GRANDE

    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

    params = {
        "depth": 5,  # tree depth
        "n_estimators": 2048,  # number of estimators / trees
        "learning_rate_weights": 0.005,  # learning rate for leaf weights
        "learning_rate_index": 0.01,  # learning rate for split indices
        "learning_rate_values": 0.01,  # learning rate for split values
        "learning_rate_leaf": 0.01,  # learning rate for leafs (logits)
        "optimizer": "adam",  # optimizer
        "cosine_decay_steps": 0,  # decay steps for lr schedule (CosineDecayRestarts)
        "loss": "crossentropy",  # loss function (default 'crossentropy' for binary & multi-class classification and 'mse' for regression)
        "focal_loss": False,  # use focal loss {True, False}
        "temperature": 0.0,  # temperature for stochastic re-weighted GD (0.0, 1.0)
        "from_logits": True,  # use logits for weighting {True, False}
        "use_class_weights": True,  # use class weights for training {True, False}
        "dropout": 0.0,  # dropout rate (here, dropout randomly disables individual estimators of the ensemble during training)
        "selected_variables": 0.8,  # feature subset percentage (0.0, 1.0)
        "data_subset_fraction": 1.0,  # data subset percentage (0.0, 1.0)
    }

    args = {
        "epochs": 1_000,  # number of epochs for training
        "early_stopping_epochs": 25,  # patience for early stopping (best weights are restored)
        "batch_size": 64,  # batch size for training
        "cat_idx": [],  # put list of categorical indices
        "objective": "binary",  # objective / task {'binary', 'classification', 'regression'}
        "random_seed": 42,
        "verbose": 0,
    }

    return GRANDE(params=params, args=args)


`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- how: avg


## Without validate

### Without fill missing data

In [11]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from utils.prepare_data import normalizeData


accuracy_score_list = []
precision_score_list = []
recall_score_list = []
auc_score_list = []
for trainPatients, testPatients in trainTest():
    dfTrain = trainPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTrain = dfTrain.drop(columns=idColumns)

    dfTest = testPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTest = dfTest.drop(columns=idColumns)

    dfTrain, dfTest, _ = normalizeData(dfTrain, dfTest)

    X_train = dfTrain.drop(columns=[labelColumn])
    y_train = dfTrain[labelColumn]

    X_test = dfTest.drop(columns=[labelColumn])
    y_test = dfTest[labelColumn]

    model = createModel()
    model.fit(X_train, y_train)

    # y_pred = model.predict(X_test)
    y_pred_proba = model.predict(X_test)[:, 1]  # For AUC

    accuracy_score_list.append(accuracy_score(y_test, np.round(y_pred_proba)))
    precision_score_list.append(precision_score(y_test, np.round(y_pred_proba)))
    recall_score_list.append(recall_score(y_test, np.round(y_pred_proba)))
    auc_score_list.append(roc_auc_score(y_test, y_pred_proba))

2024-06-28 22:23:09.798279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-28 22:23:10.793663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyError: "None of [Index(['ag', 'age', 'bg', 'bicarbonate', 'bun', 'calcium',\n       'chronic_pulmonary_disease', 'congestive_heart_failure', 'dbp', 'egfr',\n       'gcs', 'gcs_unable', 'hb', 'history_aci', 'history_ami', 'hr',\n       'hypertension', 'macroangiopathy', 'malignant_cancer',\n       'mechanical_ventilation', 'microangiopathy', 'oasis', 'phosphate',\n       'plt', 'preiculos', 'rr', 'saps2', 'sbp', 'scr', 'sofa', 'use_NaHCO3',\n       'uti', 'wbc', 'weight', 'dka_type_0', 'dka_type_1', 'dka_type_2',\n       'gender_F', 'gender_M', 'race_AMERICAN INDIAN/ALASKA NATIVE',\n       'race_ASIAN', 'race_ASIAN - CHINESE', 'race_ASIAN - SOUTH EAST ASIAN',\n       'race_BLACK/AFRICAN', 'race_BLACK/AFRICAN AMERICAN',\n       'race_BLACK/CAPE VERDEAN', 'race_BLACK/CARIBBEAN ISLAND',\n       'race_HISPANIC OR LATINO', 'race_HISPANIC/LATINO - DOMINICAN',\n       'race_HISPANIC/LATINO - GUATEMALAN', 'race_HISPANIC/LATINO - MEXICAN',\n       'race_HISPANIC/LATINO - PUERTO RICAN',\n     

In [12]:

print(f"Average AUC: {np.mean(auc_score_list)}")
print(f"Average Accuracy: {np.mean(accuracy_score_list)}")
print(f"Average Precision: {np.mean(precision_score_list)}")
print(f"Average Recall: {np.mean(recall_score_list)}")

Average AUC: nan
Average Accuracy: nan
Average Precision: nan
Average Recall: nan


/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### Fill missing with knn

In [13]:
from sklearn.metrics import roc_auc_score
from utils.prepare_data import normalizeAndFillData


accuracy_score_list_knn = []
precision_score_list_knn = []
recall_score_list_knn = []
auc_score_list_knn = []
for trainPatients, testPatients in trainTest():
    dfTrain = trainPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTrain = dfTrain.drop(columns=idColumns)

    dfTest = testPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTest = dfTest.drop(columns=idColumns)

    dfTrain, dfTest, _ = normalizeAndFillData(dfTrain, dfTest)

    X_train = dfTrain.drop(columns=[labelColumn])
    y_train = dfTrain[labelColumn]

    X_test = dfTest.drop(columns=[labelColumn])
    y_test = dfTest[labelColumn]

    model = createModel()
    model.fit(X_train, y_train)

    # y_pred = model.predict(X_test)
    y_pred_proba = model.predict(X_test)[:, 1]  # For AUC

    accuracy_score_list_knn.append(accuracy_score(y_test, np.round(y_pred_proba)))
    precision_score_list_knn.append(precision_score(y_test, np.round(y_pred_proba)))
    recall_score_list_knn.append(recall_score(y_test, np.round(y_pred_proba)))
    auc_score_list_knn.append(roc_auc_score(y_test, y_pred_proba))

KeyError: "None of [Index(['ag', 'age', 'bg', 'bicarbonate', 'bun', 'calcium',\n       'chronic_pulmonary_disease', 'congestive_heart_failure', 'dbp', 'egfr',\n       'gcs', 'gcs_unable', 'hb', 'history_aci', 'history_ami', 'hr',\n       'hypertension', 'macroangiopathy', 'malignant_cancer',\n       'mechanical_ventilation', 'microangiopathy', 'oasis', 'phosphate',\n       'plt', 'preiculos', 'rr', 'saps2', 'sbp', 'scr', 'sofa', 'use_NaHCO3',\n       'uti', 'wbc', 'weight', 'dka_type_0', 'dka_type_1', 'dka_type_2',\n       'gender_F', 'gender_M', 'race_AMERICAN INDIAN/ALASKA NATIVE',\n       'race_ASIAN', 'race_ASIAN - CHINESE', 'race_ASIAN - SOUTH EAST ASIAN',\n       'race_BLACK/AFRICAN', 'race_BLACK/AFRICAN AMERICAN',\n       'race_BLACK/CAPE VERDEAN', 'race_BLACK/CARIBBEAN ISLAND',\n       'race_HISPANIC OR LATINO', 'race_HISPANIC/LATINO - DOMINICAN',\n       'race_HISPANIC/LATINO - GUATEMALAN', 'race_HISPANIC/LATINO - MEXICAN',\n       'race_HISPANIC/LATINO - PUERTO RICAN',\n     

In [14]:

print(f"Average AUC: {np.mean(auc_score_list_knn)}")
print(f"Average Accuracy: {np.mean(accuracy_score_list_knn)}")
print(f"Average Precision: {np.mean(precision_score_list_knn)}")
print(f"Average Recall: {np.mean(recall_score_list_knn)}")

Average AUC: nan
Average Accuracy: nan
Average Precision: nan
Average Recall: nan


/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## With validate

### Without fill missing data

In [15]:
from utils.prepare_data import normalizeData


accuracy_score_list_val = []
precision_score_list_val = []
recall_score_list_val = []
auc_score_list_val = []
for trainPatients, valPatients, testPatients in trainValTest():
    dfTrain = trainPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTrain = dfTrain.drop(columns=idColumns)

    dfVal = valPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfVal = dfVal.drop(columns=idColumns)

    dfTest = testPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTest = dfTest.drop(columns=idColumns)

    dfTrain, dfTest, dfVal = normalizeData(dfTrain, dfTest, dfVal)

    X_train = dfTrain.drop(columns=[labelColumn])
    y_train = dfTrain[labelColumn]

    X_val = dfVal.drop(columns=[labelColumn]) # type: ignore
    y_val = dfVal[labelColumn] # type: ignore

    X_test = dfTest.drop(columns=[labelColumn])
    y_test = dfTest[labelColumn]

    model = createModel()
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

    # y_pred = model.predict(X_test)
    y_pred_proba = model.predict(X_test)[:, 1]  # For AUC

    accuracy_score_list_val.append(accuracy_score(y_test, np.round(y_pred_proba)))
    precision_score_list_val.append(precision_score(y_test, np.round(y_pred_proba)))
    recall_score_list_val.append(recall_score(y_test, np.round(y_pred_proba)))
    auc_score_list_val.append(roc_auc_score(y_test, y_pred_proba))

2024-06-28 22:23:44.383947: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-28 22:23:44.419797: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-28 22:23:44.420028: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

I0000 00:00:1719588230.355485 1459791 service.cc:145] XLA service 0x7f1c68004a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719588230.355511 1459791 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-06-28 22:23:50.465489: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


2024-06-28 22:23:51.158508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:470] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2024-06-28 22:23:51.158575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:474] Memory usage: 2031616 bytes free, 11539054592 bytes total.
2024-06-28 22:23:51.238081: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:470] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2024-06-28 22:23:51.238139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:474] Memory usage: 2031616 bytes free, 11539054592 bytes total.
2024-06-28 22:23:51.244815: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:580 : FAILED_PRECONDITION: DNN library initialization failed. Look at the errors above for more details.
2024-06-28 22:23:51.244856: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: FAILED_PRECONDITION: DNN library initialization failed. Look at 

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607

In [16]:


print(f"Average AUC: {np.mean(auc_score_list_val)}")
print(f"Average Accuracy: {np.mean(accuracy_score_list_val)}")
print(f"Average Precision: {np.mean(precision_score_list_val)}")
print(f"Average Recall: {np.mean(recall_score_list_val)}")

Average AUC: nan
Average Accuracy: nan
Average Precision: nan
Average Recall: nan


/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### Fill missing with knn

In [17]:
from sklearn.metrics import roc_auc_score
from utils.prepare_data import normalizeAndFillData


accuracy_score_list_val_knn = []
precision_score_list_val_knn = []
recall_score_list_val_knn = []
auc_score_list_val_knn = []
metric_dic_list_val_knn = []
for trainPatients, valPatients, testPatients in trainValTest():
    dfTrain = trainPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTrain = dfTrain.drop(columns=idColumns)

    dfVal = valPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfVal = dfVal.drop(columns=idColumns)

    dfTest = testPatients.getMeasuresBetween(
        pd.Timedelta(hours=-6), pd.Timedelta(hours=24), how
    )
    dfTest = dfTest.drop(columns=idColumns)

    dfTrain, dfTest, dfVal = normalizeAndFillData(dfTrain, dfTest, dfVal)

    X_train = dfTrain.drop(columns=[labelColumn])
    y_train = dfTrain[labelColumn]

    X_val = dfVal.drop(columns=[labelColumn])  # type: ignore
    y_val = dfVal[labelColumn]  # type: ignore

    X_test = dfTest.drop(columns=[labelColumn])
    y_test = dfTest[labelColumn]

    model = createModel()
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

    # y_pred = model.predict(X_test)
    y_pred_proba = model.predict(X_test)[:, 1]  # For AUC

    accuracy_score_list_val_knn.append(accuracy_score(y_test, np.round(y_pred_proba)))
    precision_score_list_val_knn.append(precision_score(y_test, np.round(y_pred_proba)))
    recall_score_list_val_knn.append(recall_score(y_test, np.round(y_pred_proba)))
    auc_score_list_val_knn.append(roc_auc_score(y_test, y_pred_proba))

2024-06-28 22:24:09.133370: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 35.94MiB (37683200 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:09.133450: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 32.34MiB (33914880 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:09.133511: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 29.11MiB (30523392 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:09.133573: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 26.20MiB (27471104 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:09.133631: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 23.58MiB (24724224 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:09.133688: I exter

2024-06-28 22:24:19.134258: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 35.94MiB (37683200 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:19.134355: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 35.94MiB (37683200 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-06-28 22:24:19.134368: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.11MiB (rounded to 12697600)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-06-28 22:24:19.134380: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-06-28 22:24:19.134390: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256):

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

In [18]:
print(f"Average AUC: {np.mean(auc_score_list_val_knn)}")
print(f"Average Accuracy: {np.mean(accuracy_score_list_val_knn)}")
print(f"Average Precision: {np.mean(precision_score_list_val_knn)}")
print(f"Average Recall: {np.mean(recall_score_list_val_knn)}")

Average AUC: nan
Average Accuracy: nan
Average Precision: nan
Average Recall: nan


/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
